In [1]:
import keras

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
##Import libraries
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [0]:
IMAGE_SIZE = 64
# Base model with MobileNetV2
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

In [5]:
base_model = keras.applications.VGG16(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')





58892288/58889256 [==============================] - 2s 0us/step




In [0]:
base_model.trainable = False

In [0]:
# Trainable classification head
maxpool_layer = keras.layers.GlobalMaxPooling2D()
prediction_layer = keras.layers.Dense(1, activation='sigmoid')

In [0]:
#learning_rate = 0.0001

classifier = keras.Sequential([
    base_model,
    maxpool_layer,
    prediction_layer
])

In [9]:
classifier.compile(optimizer=keras.optimizers.Adam(),
             loss='binary_crossentropy',
             metrics=['accuracy'])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
## data importing and transforming and scaling
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

In [0]:
## Scaling test data
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_set = train_datagen.flow_from_directory('/content/drive/My Drive/training',
                                               target_size=(64, 64),
                                               batch_size=6,
                                               class_mode='binary')

Found 832 images belonging to 2 classes.


In [13]:
#which is normal which is abnormal?
label_map = (train_set.class_indices)
print(label_map)

{'single lung vr abnormal': 0, 'single lung vr normal': 1}


In [14]:
## Importng test data
test_set = test_datagen.flow_from_directory('/content/drive/My Drive/test',
                                            target_size=(64, 64),
                                            batch_size=6,
                                            class_mode='binary')

Found 0 images belonging to 0 classes.


In [15]:
## fitting model to images
classifier.fit_generator(
        train_set,
        steps_per_epoch=98,
        epochs=10,
        validation_data=test_set,
        validation_steps=38)

Epoch 1/10
98/98 [==============================] - 191s 2s/step - loss: 0.7130 - acc: 0.5093
Epoch 2/10
98/98 [==============================] - 95s 967ms/step - loss: 0.6887 - acc: 0.5391
Epoch 3/10
98/98 [==============================] - 29s 297ms/step - loss: 0.6806 - acc: 0.5910
Epoch 4/10
98/98 [==============================] - 29s 297ms/step - loss: 0.6640 - acc: 0.5910
Epoch 5/10
98/98 [==============================] - 29s 299ms/step - loss: 0.6646 - acc: 0.5876
Epoch 6/10
98/98 [==============================] - 29s 298ms/step - loss: 0.6503 - acc: 0.6480
Epoch 7/10
98/98 [==============================] - 29s 297ms/step - loss: 0.6497 - acc: 0.6088
Epoch 8/10
98/98 [==============================] - 29s 298ms/step - loss: 0.6554 - acc: 0.6174
Epoch 9/10
98/98 [==============================] - 29s 300ms/step - loss: 0.6426 - acc: 0.6352
Epoch 10/10
98/98 [==============================] - 29s 299ms/step - loss: 0.6433 - acc: 0.6395


In [0]:
## prediction of single new data
import numpy as np
import keras
from keras.preprocessing import image

In [0]:
test_image= image.load_img('/content/drive/My Drive/training/single lung vr normal/3D_VR_-_SC0001_-_01-01-2000_10_20_27_0002.jpg',target_size =(64,64))

In [0]:
## Convert image to array
test_image = image.img_to_array(test_image)

In [0]:
## For single prediction change the dimension using axis. To remove problem of batch
test_image = np.expand_dims(test_image,axis = 0)
result = classifier.predict(test_image)

In [0]:
## Class of normal and abnormal

if result[0][0] == 1:
    prediction = 'normal'
else:
   prediction = 'abnormal'

In [21]:
prediction

'abnormal'